In [ ]:
import numpy as np
import autokeras as ak
import tensorflow as tf
import os 
import logging

gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
except RuntimeError as e:
    logging.info(e)

In [ ]:
training_data = np.load("../../Data/n1000000_0910_all_flat.npz")
data_all = np.column_stack([training_data['ve_dune'][:,:36], training_data['vu_dune'][:,:36], training_data['vebar_dune'][:,:36], training_data['vubar_dune'][:,:36]])

# theta13, theta23, delta
target = np.column_stack([training_data["theta13"]/180*np.pi, training_data["theta23"]/180*np.pi,
                         np.cos(training_data["delta"]/180*np.pi), np.sin(training_data["delta"]/180*np.pi)])

split = 900000
x_train = data_all[:split]
y_train = target[:split]

x_val = data_all[split:]
y_val = target[split:]

In [ ]:
dnn = ak.StructuredDataRegressor(overwrite=True, max_trials=1)
dnn.fit(x_train, y_train,
           validation_split = (x_val, y_val),
           batch_size=64,
           epochs=1,
           shuffle = True
        )

In [ ]:
model = dnn.export_model()
model.summary()

In [ ]:
index = 1
while os.path.isfile('./dnn/.h5'.format(index)):
    index += 1
model.save('./dnn/.h5'.format(index))